In [1]:
from transformers import (BitsAndBytesConfig, TrainingArguments)
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, PeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import Dataset
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import DataLoader
import transformers
import tqdm
# from trl import SFTTrainer


### Dataset Loading

In [41]:
import pandas as pd
data_train = pd.read_json("dataset/training_data.json")  # Use `lines=True` for line-delimited JSON
data_train = data_train.transpose()
data_test = pd.read_json("dataset/testing_data.json")
data_test = data_test.transpose()

In [42]:
data_train = data_train.astype(str)
data_test = data_test.astype(str)
train_dataset = Dataset.from_pandas(data_train)
test_dataset = Dataset.from_pandas(data_test)

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [43]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

base_model= AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1B",
    low_cpu_mem_usage=True,
    quantization_config=bnb_config,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [44]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
base_model.config.use_cache = False
base_model.config.pad_token_id = tokenizer.pad_token_id


base_model = prepare_model_for_kbit_training(base_model)
new_model = "llama_fine_tuned"

train_dataset = train_dataset.train_test_split(test_size=0.1)
rank = 16
alpha = rank*2

In [46]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'closest_5_queries', '__index_level_0__'],
        num_rows: 36027
    })
    test: Dataset({
        features: ['input', 'output', 'closest_5_queries', '__index_level_0__'],
        num_rows: 4003
    })
})

In [8]:
# def extract_input_and_output(data):
#     result = {}
#     for key, value in data.items():
#         print(key)
#         print(value)
#         input_text = value["input"]
#         raw_output = value["output"][0]["raw_output"]  # Assuming only one item in 'output'
#         result[input_text] = raw_output
#     return result

# # Extracted dictionary
# extracted_data = extract_input_and_output(train_dataset)

### LoRA Config

In [ ]:
peft_config = LoraConfig(
    r=rank,
    lora_alpha=rank,
    lora_dropout=0,
    bias="none",
    task_type="CAUSAL_LM",
# target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"]
)

model = get_peft_model(base_model, peft_config)

def train(
    model: PeftModelForCausalLM,
    tokenizer: AutoTokenizer,
    train_dataset: Dataset,
    grad_acc_steps: int = 1,
    batch_size: int = 32,
    epochs: int = 1,
) -> None:
    tokenized_dataset = train_dataset.map(
        lambda x: {
            "input_ids": tokenizer.encode(x["input"] + x["output"])
            + [tokenizer.eos_token_id]
        }
    )
    # .remove_columns(["text", "target", "label"])

    dataloader = DataLoader(
        tokenized_dataset,
        batch_size=batch_size,
        collate_fn=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
        shuffle=True,
    )
    opt = torch.optim.AdamW(model.parameters(), lr=4e-4)
    step = 0
    for epoch_num in range(epochs):
        for batch in (pbar := tqdm(dataloader, desc=f"epoch {epoch_num+1}/{epochs}")):
            batch = {k: v.cuda() for k, v in batch.items()}
            outputs = model(**batch)
            outputs["loss"].backward()
            print("Loss: ", outputs["loss"])

            if (step + 1) % grad_acc_steps == 0:
                opt.step()
                opt.zero_grad()

            pbar.set_postfix({"loss": outputs["loss"].item()})
            step += 1

    model.save_pretrained("llama3.2-1B-quantized-lora")



In [ ]:
train(model, tokenizer, train_dataset)

Map:   0%|          | 0/36027 [00:00<?, ? examples/s]

Map:   0%|          | 0/4003 [00:00<?, ? examples/s]

### Supervised Fine-tuning

In [ ]:
# training_arguments = TrainingArguments(
#     output_dir=new_model,
#     per_device_train_batch_size=6,
#     per_device_eval_batch_size=6,
#     log_level="debug",
#     gradient_accumulation_steps=6,
#     optim="paged_adamw_32bit",
#     num_train_epochs=3,
#     evaluation_strategy="steps",
#     eval_steps=6,
#     logging_steps=6,
#     save_steps=6,
#     warmup_steps=10,
#     logging_strategy="steps",
#     learning_rate=3e-4,
#     warmup_ratio=0.1,
#     load_best_model_at_end=True,
#     overwrite_output_dir=True,
#     fp16=True,
#     bf16=False,
#     group_by_length=True,
#     lr_scheduler_type="linear",
#     report_to="none",
# )

# trainer = SFTTrainer(
#     model=base_model,
#     train_dataset=train_dataset["train"],
#     eval_dataset=train_dataset["test"],
#     peft_config=peft_config,
#     max_seq_length=3000,
#     dataset_text_field="texts",
#     tokenizer=tokenizer,
#     args=training_arguments,
#     packing= False,
# )

# initial_eval_values = trainer.evaluate()
# print(initial_eval_values)
# initial_eval_loss = initial_eval_values['eval_loss']
# trainer.train()

# training_loss_history = []
# eval_loss_history = [initial_eval_loss]
# for step in trainer.state.log_history:
#   if 'loss' in step:
#     training_loss_history.append(step['loss'])
#   elif "eval_loss" in step:
#     eval_loss_history.append(step['eval_loss'])

# print(training_loss_history)
# print(eval_loss_history)